In [ ]:
import os
from pathlib import Path
from tqdm import tqdm

# **1.- Extracción de características utilizando DinoV2 (vision transformer)**

Para comenzar con la extracción de características utilizando DINOv2 (Vision Transformer), seguimos los siguientes pasos:

- Preparación del entorno: Asegurarse de tener instalado Python y las bibliotecas necesarias --> PyTorch, torchvision, y DINOv2.  Asegurarse que los frames de las especies (test, train, val) estén organizados en carpetas correspondientes.

- Preprocesamiento de imágenes: Las imágenes deben ser redimensionadas a 224x224 píxeles y normalizadas con los valores de media y desviación estándar esperados por DINOv2.

- Extracción de características con DINOv2: Utilizaremos el modelo preentrenado ViT-b/14 para generar embeddings. Los pasos incluyen cargar las imágenes, preprocesarlas, y pasar cada una por el modelo para obtener vectores de características.


## DINOv2 Installation

In [ ]:
!unzip dinov2-main.zip

In [ ]:
!pip install -r /content/dinov2-main/requirements.txt

In [ ]:
import sys
sys.path.append("/content/dinov2-main")

In [ ]:
!pip install -r /content/dinov2-main/requirements-dev.txt

In [ ]:
import sys
sys.path.append("/content/dinov2-main/dinov2/hub")

## Función para carga de imágenes

In [ ]:
import pandas as pd

# Cargar el archivo XLSX
metadata_file = "/home/rmunoz/Cropped_images/cropped_images_metadata.xlsx"
df = pd.read_excel(metadata_file)

# Extraer los paths de las imágenes
image_paths = df["Cropped Image Path"].tolist()

## Carga del modelo

In [ ]:
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
from dinov2.models import vision_transformer
from dinov2.hub.backbones import dinov2_vitb14, dinov2_vits14

In [ ]:
# Definir el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi  -L

In [ ]:
import torch
device_id = 1  # Cambiar a otro índice de GPU disponible
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
# Cargar el modelo DINOv2 preentrenado
model = dinov2_vitb14().to(device)  # Mover el modelo al dispositivo especificado
model.eval()

## Preprocesamiento de las imágenes

In [ ]:
# Definir las transformaciones de imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
])

## Función para la extracción de características

In [ ]:
# Función para procesar las imágenes y extraer características
def extract_features(image_paths, model, transform, device):
    features = []
    for path in tqdm(image_paths, desc = "Extracting features"):
        try:
            image = Image.open(path).convert("RGB")
            input_tensor = transform(image).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(input_tensor)
            features.append(output.cpu().numpy().flatten())
        except Exception as e:
            print(f"Error with {path}: {e}")
    return np.array(features)

In [ ]:
from pathlib import Path

# Filtrar únicamente los paths existentes
df = df[df["Cropped Image Path"].apply(lambda x: Path(x).exists())]

# Agrupar por partición
train_paths = df[df["Partition"] == "train"]["Cropped Image Path"].tolist()
val_paths = df[df["Partition"] == "validation"]["Cropped Image Path"].tolist()
test_paths = df[df["Partition"] == "test"]["Cropped Image Path"].tolist()

In [ ]:
# Extraer las características por separado
features_train = extract_features(train_paths, model, transform, device)
features_val = extract_features(val_paths, model, transform, device)
features_test = extract_features(test_paths, model, transform, device)

In [ ]:
five_rows = features_train[:5]
five_rows.max()

In [ ]:
features_train.max()

In [ ]:
features_train.min()

Se extraen las características de las imágenes por separado (train, val y test) y se guardan en los archivos features_train.npy, features_val.npy y features_test.npy respectivamente.

Posteriormente estos archivos se utilizarán para aplicar los algoritmos de clusterización, como K-Means y GMM.

In [ ]:
# Se guardan las características para un posterior agrupamiento (clustering)
np.save("/content/features_train.npy", features_train)
np.save("/content/features_val.npy", features_val)
np.save("/content/features_test.npy", features_test)

# **2.- Clusterización:**

A continuación se aplica algoritmos de clusterización como K-Means y Gaussian Mixture Models (GMM) sobre los vectores de características extraídos.
Se probarán diferentes configuraciones para optimizar el número de clusters.

## Algoritmo K-Means

In [ ]:
from google.colab import files
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances_argmin_min

# Clustering K-Means
k_values = range(2, 11) # Se prueba con k = 2 hasta k = 10
inertias = []

for k in k_values:
    kmeans = KMeans(n_clusters = k, random_state = 0, n_init = 'auto')
    kmeans.fit(features_train)
    _, distances = pairwise_distances_argmin_min(features_val, kmeans.cluster_centers_)
    inertia = np.sum(distances ** 2)
    inertias.append(inertia)

plt.figure(figsize = (8, 5))
plt.plot(k_values, inertias, marker = 'o')
plt.title("Inercia en conjunto de validación vs número de clusters")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia (validación)")
plt.grid(True)

# Guardar la figura
plt.savefig("inercia_vs_k.png", dpi = 600, bbox_inches = 'tight')
files.download("inercia_vs_k.png")

plt.show()

### Visualizamos y seleccionamos el mejor valor de k y entrenamos el modelo K-Means

In [ ]:
# Definición de k óptimo según visualización del codo
optimal_k = 5

# Entrenamiento final con k óptimo
kmeans_final = KMeans(n_clusters = optimal_k, random_state = 0, n_init = 'auto')
kmeans_final.fit(features_train)

# Predicción en conjunto de test
test_preds5 = kmeans_final.predict(features_test)

In [ ]:
# Guardar las asignaciones de clusters
clusters_kmeans = test_preds5
output_path_clustersKmeans = "/content/clusters_kmeans5.npy"
np.save(output_path_clustersKmeans, clusters_kmeans)

In [ ]:
# Distribución de clusters en el conjunto de test
unique, counts = np.unique(test_preds5, return_counts = True)
kmeans_distribution = dict(zip(unique, counts))

print("\nDistribución de imágenes por cluster en TEST:")
for cluster, count in kmeans_distribution.items():
    print(f"Cluster {cluster}: {count} imágenes")

plt.figure(figsize = (8, 5))
plt.bar(kmeans_distribution.keys(), kmeans_distribution.values(), color = 'blue')
plt.xlabel("Cluster")
plt.ylabel("Cantidad de imágenes")
plt.title("Distribución de imágenes por cluster (test)")
plt.grid(axis = 'y')
plt.xticks(list(kmeans_distribution.keys()))

# Guardar la figura
plt.savefig("distribucion_clusters_test_k5.png", dpi = 600, bbox_inches = 'tight')
files.download("distribucion_clusters_test_k5.png")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

images_per_cluster = 20

print("\nEjemplos aleatorios por cluster:")
for cluster_id in range(optimal_k):
    print(f"\nCluster {cluster_id}:")
    cluster_indices = np.where(test_preds5 == cluster_id)[0]
    if len(cluster_indices) == 0:
        print("No hay imágenes en este cluster.")
        continue

    n_images = min(images_per_cluster, len(cluster_indices))
    random_samples = random.sample(list(cluster_indices), n_images)

    # Crear grilla para visualizar
    cols = 4
    rows = (n_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize = (cols * 3, rows * 3))

    for i, idx in enumerate(random_samples):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        try:
            img = Image.open(test_paths[idx])
            ax.imshow(img.resize((224, 224)))
            ax.axis('off')
        except:
            ax.set_title("Error")
            ax.axis('off')

    # Rellenar celdas vacías si las hay
    for i in range(n_images, rows * cols):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        ax.axis('off')

    plt.suptitle(f"Cluster {cluster_id} - {n_images} muestras", fontsize = 14)
    plt.tight_layout()

    # Guardar la figura
    output_path = f"cluster_{cluster_id}_muestras.png"
    plt.savefig(output_path, dpi = 600, bbox_inches = 'tight')

    from google.colab import files
    files.download(output_path)

    plt.show()

    plt.close()

In [ ]:
# Inercia en validación después de predict
val_preds = kmeans_final.predict(features_val)
_, val_distances = pairwise_distances_argmin_min(features_val, kmeans_final.cluster_centers_)
val_inertia = np.sum(val_distances ** 2)
print(f"\nInercia en VALIDACIÓN con k = {optimal_k}: {val_inertia:.2f}")

In [ ]:
print("Shape de features_val:", features_val.shape)
print("Cantidad de imágenes en validación:", len(val_paths))

In [ ]:
from sklearn.metrics import confusion_matrix, jaccard_score, adjusted_rand_score, fowlkes_mallows_score
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

# Matriz de confusión
y_true = df[df["Partition"] == "test"]["Species"].tolist()
le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true)
y_pred_kmeans = test_preds5

cm_kmeans = confusion_matrix(y_true_encoded, y_pred_kmeans)

plt.figure(figsize = (10, 6))
sns.heatmap(cm_kmeans, annot = True, fmt = 'd', cmap = 'Purples', xticklabels = range(optimal_k), yticklabels = le.classes_)
plt.xlabel("Cluster inferido (KMeans)")
plt.ylabel("Etiqueta verdadera (especies)")
plt.title("Matriz de confusión - KMeans vs etiquetas reales")

# Guardar la matriz de confusión
plt.savefig("matriz_confusion_kmeans5.png", dpi = 600, bbox_inches = 'tight')
files.download("matriz_confusion_kmeans5.png")

plt.show()

In [ ]:
# Asignación de clase dominante por cluster y Jaccard
assignments_kmeans = {}
for cluster_id in range(optimal_k):
    indices = np.where(np.array(y_pred_kmeans) == cluster_id)[0]
    true_labels_in_cluster = [y_true_encoded[i] for i in indices]
    if true_labels_in_cluster:
        majority_label = max(set(true_labels_in_cluster), key = true_labels_in_cluster.count)
        assignments_kmeans[cluster_id] = majority_label
    else:
        assignments_kmeans[cluster_id] = -1

y_pred_mapped_kmeans = [assignments_kmeans[c] for c in y_pred_kmeans]
jaccard_macro_kmeans = jaccard_score(y_true_encoded, y_pred_mapped_kmeans, average = 'macro')
print(f"Jaccard Index (macro) - KMeans: {jaccard_macro_kmeans:.4f}")

In [ ]:
# Métricas adicionales: ARI y Fowlkes-Mallows
ari_kmeans = adjusted_rand_score(y_true_encoded, y_pred_kmeans)
fmi_kmeans = fowlkes_mallows_score(y_true_encoded, y_pred_kmeans)
print(f"Adjusted Rand Index (ARI) - KMeans: {ari_kmeans:.4f}")
print(f"Fowlkes-Mallows Index - KMeans: {fmi_kmeans:.4f}")

In [ ]:
# Visualización comparativa con t-SNE
fig, axes = plt.subplots(1, 2, figsize = (16, 6))

# t-SNE con etiquetas reales
scatter1 = axes[0].scatter(tsne_result[:, 0], tsne_result[:, 1],
                           c = y_true_encoded, cmap = "tab20", s = 4)
axes[0].set_title("t-SNE (test) - Etiquetas reales")
axes[0].grid(True)

# t-SNE con clusters de KMeans
scatter2 = axes[1].scatter(tsne_result[:, 0], tsne_result[:, 1],
                           c = y_pred_kmeans, cmap = "tab10", s = 4)
axes[1].set_title("t-SNE (test) - Clusters inferidos por KMeans")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

unique_clusters = np.unique(y_pred_kmeans)
unique_labels = np.unique(y_true_encoded)
markers = ['o', 's', 'v', '^', '<', '>', 'd', '*', 'P', 'X']

plt.figure(figsize = (10, 7))
for cluster_id in unique_clusters:
    for label_id in unique_labels:
        indices = np.where((y_pred_kmeans == cluster_id) & (y_true_encoded == label_id))[0]
        if len(indices) == 0:
            continue
        plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1],
                    label = f"Sp {label_id} / Cl {cluster_id}",
                    s = 8,
                    c = [plt.cm.tab20(label_id / len(unique_labels))],
                    marker = markers[cluster_id % len(markers)],
                    edgecolors = 'none')

plt.title("t-SNE (test) - Color: Especie real | Forma: Cluster KMeans")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.legend(markerscale = 1.5, fontsize = 8, bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.tight_layout()
plt.show()

## Agrupamos por combinación especie/cluster y graficamos con centroides

En este gráfico se agrupa todos los puntos con la misma especie y cluster, y se coloca un número en el centroide del grupo.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Nombres reales de especies (orden según codificación)
nombres_especies = {
    0: 'Tinamú',
    1: 'Agutí',
    2: 'Paca',
    3: 'Armadillo',
    4: 'Roedor',
    5: 'Ardilla'
}

# Crear DataFrame con t-SNE + predicciones
df_tsne = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'especie': y_true_encoded,
    'cluster': y_pred_kmeans
})

plt.figure(figsize = (12, 8))

# Colores por especie real
colores = plt.cm.Dark2(np.linspace(0, 1, len(nombres_especies)))

for idx, especie in enumerate(df_tsne['especie'].unique()):
    df_subset = df_tsne[df_tsne['especie'] == especie]
    plt.scatter(df_subset['x'], df_subset['y'],
                s = 12, alpha = 0.5, color = colores[idx],
                label=nombres_especies.get(especie, f"Especie {especie}"))

# Colocar número del cluster en el centro de cada grupo
for (especie, cluster), group in df_tsne.groupby(['especie', 'cluster']):
    x_med = group['x'].mean()
    y_med = group['y'].mean()
    plt.text(x_med, y_med, str(cluster),
             fontsize = 13, weight = 'bold', ha = 'center', va = 'center',
             color = 'black',
             bbox = dict(facecolor = 'white', alpha = 0.8, edgecolor = 'gray', boxstyle = 'circle'))

plt.title("t-SNE — Color: Especie real  |  Número: Cluster K-Means (k=6)", fontsize = 14)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title = "Especie real", bbox_to_anchor = (1.05, 1), loc = 'upper left', fontsize = 9)
plt.tight_layout()

# Guardar la figura
plt.savefig("tsne_especie-cluster_KMEANS_5.png", dpi = 600, bbox_inches = 'tight')
files.download("tsne_especie-cluster_KMEANS_5.png")

plt.show()


### Probamos con k = 6

In [ ]:
optimal_k = 6

# Entrenamiento final con k óptimo
kmeans_final = KMeans(n_clusters = optimal_k, random_state = 0, n_init = 'auto')
kmeans_final.fit(features_train)

# Predicción en conjunto de test
test_preds6 = kmeans_final.predict(features_test)

In [ ]:
# Guardar las asignaciones de clusters
clusters_kmeans = test_preds6
output_path_clustersKmeans = "/content/clusters_kmeans6.npy"
np.save(output_path_clustersKmeans, clusters_kmeans)

In [ ]:
# Distribución de clusters en el conjunto de test
unique, counts = np.unique(test_preds6, return_counts = True)
kmeans_distribution = dict(zip(unique, counts))

print("\nDistribución de imágenes por cluster en TEST:")
for cluster, count in kmeans_distribution.items():
    print(f"Cluster {cluster}: {count} imágenes")

plt.figure(figsize = (8, 5))
plt.bar(kmeans_distribution.keys(), kmeans_distribution.values(), color = 'blue')
plt.xlabel("Cluster")
plt.ylabel("Cantidad de imágenes")
plt.title("Distribución de imágenes por cluster (test)")
plt.grid(axis = 'y')
plt.xticks(list(kmeans_distribution.keys()))

# Guardar la figura
plt.savefig("distribucion_clusters_test_k6.png", dpi = 600, bbox_inches = 'tight')
files.download("distribucion_clusters_test_k6.png")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

images_per_cluster = 20

print("\nEjemplos aleatorios por cluster:")
for cluster_id in range(optimal_k):
    print(f"\nCluster {cluster_id}:")
    cluster_indices = np.where(test_preds6 == cluster_id)[0]
    if len(cluster_indices) == 0:
        print("No hay imágenes en este cluster.")
        continue

    n_images = min(images_per_cluster, len(cluster_indices))
    random_samples = random.sample(list(cluster_indices), n_images)

    # Grilla para visualizar
    cols = 4
    rows = (n_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize = (cols * 3, rows * 3))

    for i, idx in enumerate(random_samples):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        try:
            img = Image.open(test_paths[idx])
            ax.imshow(img.resize((224, 224)))
            ax.axis("off")
        except:
            ax.set_title("Error")
            ax.axis("off")

    # Rellenar celdas vacías si las hay
    for i in range(n_images, rows * cols):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        ax.axis("off")

    plt.suptitle(f"Cluster {cluster_id} - {n_images} muestras", fontsize = 14)
    plt.tight_layout()

    # Guardar la figura
    output_path = f"cluster_{cluster_id}_muestras.png"
    plt.savefig(output_path, dpi = 600, bbox_inches = 'tight')

    from google.colab import files
    files.download(output_path)

    plt.show()

    plt.close()

In [ ]:
# Inercia en validación después de predict
val_preds = kmeans_final.predict(features_val)
_, val_distances = pairwise_distances_argmin_min(features_val, kmeans_final.cluster_centers_)
val_inertia = np.sum(val_distances ** 2)
print(f"\nInercia en VALIDACIÓN con k = {optimal_k}: {val_inertia:.2f}")

In [ ]:
print("Shape de features_val:", features_val.shape)
print("Cantidad de imágenes en validación:", len(val_paths))

In [ ]:
# Matriz de confusión
y_true = df[df["Partition"] == "test"]["Species"].tolist()
le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true)
y_pred_kmeans = test_preds6

cm_kmeans = confusion_matrix(y_true_encoded, y_pred_kmeans)

plt.figure(figsize = (10, 6))
sns.heatmap(cm_kmeans, annot = True, fmt = 'd', cmap = 'Purples', xticklabels = range(optimal_k), yticklabels = le.classes_)
plt.xlabel("Cluster inferido (KMeans)")
plt.ylabel("Etiqueta verdadera (especies)")
plt.title("Matriz de confusión - KMeans vs etiquetas reales")

# Guardar la matriz de confusión
plt.savefig("matriz_confusion_kmeans6.png", dpi = 600, bbox_inches = 'tight')
files.download("matriz_confusion_kmeans6.png")

plt.show()

In [ ]:
# Asignación de clase dominante por cluster y Jaccard
assignments_kmeans = {}
for cluster_id in range(optimal_k):
    indices = np.where(np.array(y_pred_kmeans) == cluster_id)[0]
    true_labels_in_cluster = [y_true_encoded[i] for i in indices]
    if true_labels_in_cluster:
        majority_label = max(set(true_labels_in_cluster), key = true_labels_in_cluster.count)
        assignments_kmeans[cluster_id] = majority_label
    else:
        assignments_kmeans[cluster_id] = -1

y_pred_mapped_kmeans = [assignments_kmeans[c] for c in y_pred_kmeans]
jaccard_macro_kmeans = jaccard_score(y_true_encoded, y_pred_mapped_kmeans, average = 'macro')
print(f"Jaccard Index (macro) - KMeans: {jaccard_macro_kmeans:.4f}")

In [ ]:
# Métricas adicionales: ARI y Fowlkes-Mallows
ari_kmeans = adjusted_rand_score(y_true_encoded, y_pred_kmeans)
fmi_kmeans = fowlkes_mallows_score(y_true_encoded, y_pred_kmeans)
print(f"Adjusted Rand Index (ARI) - KMeans: {ari_kmeans:.4f}")
print(f"Fowlkes-Mallows Index - KMeans: {fmi_kmeans:.4f}")

In [ ]:
# Visualización comparativa con t-SNE
fig, axes = plt.subplots(1, 2, figsize = (16, 6))

# t-SNE con etiquetas reales
scatter1 = axes[0].scatter(tsne_result[:, 0], tsne_result[:, 1],
                           c = y_true_encoded, cmap = 'tab20', s = 4)
axes[0].set_title("t-SNE (test) - Etiquetas reales")
axes[0].grid(True)

# t-SNE con clusters de KMeans
scatter2 = axes[1].scatter(tsne_result[:, 0], tsne_result[:, 1],
                           c = y_pred_kmeans, cmap = 'tab10', s = 4)
axes[1].set_title("t-SNE (test) - Clusters inferidos por KMeans")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

unique_clusters = np.unique(y_pred_kmeans)
unique_labels = np.unique(y_true_encoded)
markers = ['o', 's', 'v', '^', '<', '>', 'd', '*', 'P', 'X']

plt.figure(figsize = (10, 7))
for cluster_id in unique_clusters:
    for label_id in unique_labels:
        indices = np.where((y_pred_kmeans == cluster_id) & (y_true_encoded == label_id))[0]
        if len(indices) == 0:
            continue
        plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1],
                    label = f"Sp {label_id} / Cl {cluster_id}",
                    s = 8,
                    c = [plt.cm.tab20(label_id / len(unique_labels))],
                    marker = markers[cluster_id % len(markers)],
                    edgecolors = 'none')

plt.title("t-SNE (test) - Color: Especie real | Forma: Cluster KMeans")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.legend(markerscale = 1.5, fontsize = 8, bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Nombres reales de especies (orden según codificación)
nombres_especies = {
    0: 'Tinamú',
    1: 'Agutí',
    2: 'Paca',
    3: 'Armadillo',
    4: 'Roedor',
    5: 'Ardilla'
}

# Crear DataFrame con t-SNE + predicciones
df_tsne = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'especie': y_true_encoded,
    'cluster': y_pred_kmeans
})

plt.figure(figsize = (12, 8))

# Colores por especie real
colores = plt.cm.Dark2(np.linspace(0, 1, len(nombres_especies)))

for idx, especie in enumerate(df_tsne['especie'].unique()):
    df_subset = df_tsne[df_tsne['especie'] == especie]
    plt.scatter(df_subset['x'], df_subset['y'],
                s = 12, alpha = 0.5, color = colores[idx],
                label=nombres_especies.get(especie, f"Especie {especie}"))

# Colocar número del cluster en el centro de cada grupo
for (especie, cluster), group in df_tsne.groupby(['especie', 'cluster']):
    x_med = group['x'].mean()
    y_med = group['y'].mean()
    plt.text(x_med, y_med, str(cluster),
             fontsize = 13, weight = 'bold', ha = 'center', va = 'center',
             color = 'black',
             bbox = dict(facecolor = 'white', alpha = 0.8, edgecolor = 'gray', boxstyle = 'circle'))

plt.title("t-SNE — Color: Especie real  |  Número: Cluster K-Means (k=6)", fontsize = 14)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title = "Especie real", bbox_to_anchor = (1.05, 1), loc = 'upper left', fontsize = 9)
plt.tight_layout()

# Guardar la figura
plt.savefig("tsne_especie-cluster_KMEANS_6.png", dpi = 600, bbox_inches = 'tight')
files.download("tsne_especie-cluster_KMEANS_6.png")

plt.show()

**Silhouette score**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np

silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters = k, random_state = 42, n_init = 10)
    kmeans.fit(features_train)
    val_labels = kmeans.predict(features_val)
    score = silhouette_score(features_val, val_labels)
    silhouette_scores.append(score)

plt.figure(figsize = (8, 5))
plt.plot(K_range, silhouette_scores, marker = 'o', color = 'green')
plt.title("Silhouette score en validación vs número de clusters (K)")
plt.xlabel("Número de Clusters (K)")
plt.ylabel("Silhouette Score Promedio")
plt.grid(True)
plt.tight_layout()

# Guardar la figura
plt.savefig("KMEANS_Silhouette_score.png", dpi = 600, bbox_inches = 'tight')
files.download("KMEANS_Silhouette_score.png")

plt.show()

## Algoritmo GMM

In [ ]:
# Clustering con GMM
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from IPython.display import display

# Selección automática del mejor k para GMM (AIC/BIC)
k_values = range(1, 11)
aic = []
bic = []

for k in k_values:
    gmm_k = GaussianMixture(n_components = k, covariance_type = "full", random_state = 42)
    gmm_k.fit(features_train)
    aic.append(gmm_k.aic(features_train))
    bic.append(gmm_k.bic(features_train))


# Visualización
plt.figure(figsize = (10, 5))
plt.plot(k_values, aic, label = "AIC", marker = "o")
plt.plot(k_values, bic, label = "BIC", marker = "o")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Score")
plt.title("Selección de número de clusters para GMM")
plt.legend()
plt.grid(True)

# Guardar la figura
plt.savefig("GMM_AIC_&_BIC_score.png", dpi = 600, bbox_inches = 'tight')
files.download("GMM_AIC_&_BIC_score.png")

plt.show()

# Selección automática del mejor k por BIC
optimal_k_gmm = k_values[np.argmin(bic)]
print(f"Número óptimo de clusters según BIC: {optimal_k_gmm}")

In [ ]:
# Entrenamiento final de GMM y predicción
gmm = GaussianMixture(n_components = optimal_k_gmm, covariance_type = "full", random_state = 0)
gmm.fit(features_train)
gmm_preds_test3 = gmm.predict(features_test)

In [ ]:
# Guardar las asignaciones de clusters
clusters_GMM = gmm_preds_test3
output_path_clustersGMM = "/content/clusters_gmm3.npy"
np.save(output_path_clustersGMM, clusters_GMM)

In [ ]:
# Distribución de clusters en el conjunto de test
unique_k3, counts_k3 = np.unique(gmm_preds_test3, return_counts = True)
distribution_k3 = dict(zip(unique_k3, counts_k3))
print("Distribución de imágenes por cluster en TEST (GMM k = 3):")
for cluster, count in distribution_k3.items():
    print(f"Cluster {cluster}: {count} imágenes")

plt.figure(figsize = (8, 5))
plt.bar(distribution_k3.keys(), distribution_k3.values(), color = 'tomato')
plt.xlabel("Cluster GMM")
plt.ylabel("Cantidad de imágenes")
plt.title("Distribución de imágenes por cluster (test) - GMM k = 3")
plt.grid(axis = 'y')
plt.xticks(list(distribution_k3.keys()))
plt.tight_layout()

# Guardar la figura
plt.savefig("distribucion_clusters_test_GMM_k3.png", dpi = 600, bbox_inches = 'tight')
files.download("distribucion_clusters_test_GMM_k3.png")

plt.show()

In [ ]:
import random
from PIL import Image

images_per_cluster = 20

for cluster_id in range(optimal_k_gmm):
    print(f"\nCluster {cluster_id}:")
    cluster_indices = np.where(gmm_preds_test3 == cluster_id)[0]
    if len(cluster_indices) == 0:
        print("No hay imágenes en este cluster.")
        continue

    n_images = min(images_per_cluster, len(cluster_indices))
    random_samples = random.sample(list(cluster_indices), n_images)

    cols = 4
    rows = (n_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))

    for i, idx in enumerate(random_samples):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        try:
            img = Image.open(test_paths[idx])
            ax.imshow(img.resize((224, 224)))
            ax.axis('off')
        except:
            ax.set_title("Error")
            ax.axis('off')

    for i in range(n_images, rows * cols):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        ax.axis('off')

    plt.suptitle(f"GMM k=3 - Cluster {cluster_id}", fontsize = 14)
    plt.tight_layout()

    # Guardar la figura
    output_path = f"cluster_{cluster_id}_muestras.png"
    plt.savefig(output_path, dpi = 300, bbox_inches = 'tight')

    from google.colab import files
    files.download(output_path)

    plt.show()

    plt.close()


In [ ]:
# Inercia en validación después de predict
from sklearn.metrics import pairwise_distances_argmin_min

gmm_val_preds3 = gmm.predict(features_val)
_, val_distances_k3 = pairwise_distances_argmin_min(features_val, gmm.means_)
val_inertia_k3 = np.sum(val_distances_k3 ** 2)
print(f"Inercia en VALIDACIÓN con GMM k = 3: {val_inertia_k3:.2f}")


In [ ]:
# Matriz de confusión
y_true = df[df["Partition"] == "test"]["Species"].tolist()
le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true)
y_pred = gmm_preds_test3

cm = confusion_matrix(y_true_encoded, y_pred)

plt.figure(figsize = (10, 6))
sns.heatmap(cm, annot = True, fmt = "d", cmap = "Blues", xticklabels = range(optimal_k_gmm), yticklabels = le.classes_)
plt.xlabel("Cluster inferido (GMM)")
plt.ylabel("Etiqueta verdadera (especies)")
plt.title("Matriz de confusión - GMM vs etiquetas reales")

# Guardar la matriz de confusión
plt.savefig("matriz_confusion_GMM3.png", dpi = 600, bbox_inches = 'tight')
files.download("matriz_confusion_GMM3.png")

plt.show()

In [ ]:
# Asignación de clase dominante por cluster y Jaccard
assignments = {}
for cluster_id in range(optimal_k_gmm):
    indices = np.where(np.array(y_pred) == cluster_id)[0]
    true_labels_in_cluster = [y_true_encoded[i] for i in indices]
    if true_labels_in_cluster:
        majority_label = max(set(true_labels_in_cluster), key = true_labels_in_cluster.count)
        assignments[cluster_id] = majority_label
    else:
        assignments[cluster_id] = -1

y_pred_mapped = [assignments[c] for c in y_pred]
jaccard_macro = jaccard_score(y_true_encoded, y_pred_mapped, average = "macro")
print(f"Jaccard Index (macro): {jaccard_macro:.4f}")

In [ ]:
# Métricas adicionales: ARI y Fowlkes-Mallows
ari = adjusted_rand_score(y_true_encoded, y_pred)
fmi = fowlkes_mallows_score(y_true_encoded, y_pred)
print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Fowlkes-Mallows Index: {fmi:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

unique_clusters_gmm = np.unique(y_pred)
unique_labels = np.unique(y_true_encoded)
markers = ["o", "s", "v", "^", "<", ">", "d", "*", "P", "X"]

plt.figure(figsize = (10, 7))

for cluster_id in unique_clusters_gmm:
    for label_id in unique_labels:
        indices = np.where((y_pred == cluster_id) & (y_true_encoded == label_id))[0]
        if len(indices) == 0:
            continue
        plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1],
                    label = f"Sp {label_id} / Cl {cluster_id}",
                    s = 8,
                    c = [plt.cm.tab20(label_id / len(unique_labels))],
                    marker = markers[int(cluster_id) % len(markers)],
                    edgecolors = "none")

plt.title("t-SNE (test) - Color: Especie real | Forma: Cluster GMM")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.legend(markerscale = 1.5, fontsize = 8, bbox_to_anchor = (1.05, 1), loc = "upper left")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Nombres reales de especies (orden según codificación)
nombres_especies = {
    0: 'Tinamú',
    1: 'Agutí',
    2: 'Paca',
    3: 'Armadillo',
    4: 'Roedor',
    5: 'Ardilla'
}

# Crear DataFrame con t-SNE + predicciones
df_tsne = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'especie': y_true_encoded,
    'cluster': y_pred
})

plt.figure(figsize = (12, 8))

# Colores por especie real
colores = plt.cm.Dark2(np.linspace(0, 1, len(nombres_especies)))

for idx, especie in enumerate(df_tsne['especie'].unique()):
    df_subset = df_tsne[df_tsne['especie'] == especie]
    plt.scatter(df_subset['x'], df_subset['y'],
                s = 12, alpha = 0.5, color = colores[idx],
                label=nombres_especies.get(especie, f"Especie {especie}"))

# Colocar número del cluster en el centro de cada grupo
for (especie, cluster), group in df_tsne.groupby(['especie', 'cluster']):
    x_med = group['x'].mean()
    y_med = group['y'].mean()
    plt.text(x_med, y_med, str(cluster),
             fontsize = 13, weight = 'bold', ha = 'center', va = 'center',
             color = 'black',
             bbox = dict(facecolor = 'white', alpha = 0.8, edgecolor = 'gray', boxstyle = 'circle'))

plt.title("t-SNE — Color: Especie real  |  Número: Cluster K-Means (k=6)", fontsize = 14)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title = "Especie real", bbox_to_anchor = (1.05, 1), loc = 'upper left', fontsize = 9)
plt.tight_layout()

# Guardar la figura
plt.savefig("tsne_especie-cluster_GMM_3.png", dpi = 600, bbox_inches = 'tight')
files.download("tsne_especie-cluster_GMM_3.png")

plt.show()

### Con k = 6

In [ ]:
# Entrenamiento final de GMM y predicción
optimal_k_gmm = 6
gmm = GaussianMixture(n_components = optimal_k_gmm, covariance_type = "full", random_state = 0)
gmm.fit(features_train)
gmm_preds_test6 = gmm.predict(features_test)

In [ ]:
# Guardar las asignaciones de clusters
clusters_GMM = gmm_preds_test6
output_path_clustersGMM = "/content/clusters_gmm6.npy"
np.save(output_path_clustersGMM, clusters_GMM)

In [ ]:
# Distribución de clusters en el conjunto de test
unique_k6, counts_k6 = np.unique(gmm_preds_test6, return_counts = True)
distribution_k6 = dict(zip(unique_k6, counts_k6))
print("Distribución de imágenes por cluster en TEST (GMM k = 6):")
for cluster, count in distribution_k6.items():
    print(f"Cluster {cluster}: {count} imágenes")

plt.figure(figsize = (8, 5))
plt.bar(distribution_k6.keys(), distribution_k6.values(), color = 'seagreen')
plt.xlabel("Cluster GMM")
plt.ylabel("Cantidad de imágenes")
plt.title("Distribución de imágenes por cluster (test) - GMM k = 6")
plt.grid(axis='y')
plt.xticks(list(distribution_k6.keys()))
plt.tight_layout()

# Guardar la figura
plt.savefig("distribucion_clusters_test_GMM_k6.png", dpi = 600, bbox_inches = 'tight')
files.download("distribucion_clusters_test_GMM_k6.png")

plt.show()

In [ ]:
images_per_cluster = 20

for cluster_id in range(optimal_k_gmm):
    print(f"\nCluster {cluster_id}:")
    cluster_indices = np.where(gmm_preds_test6 == cluster_id)[0]
    if len(cluster_indices) == 0:
        print("No hay imágenes en este cluster.")
        continue

    n_images = min(images_per_cluster, len(cluster_indices))
    random_samples = random.sample(list(cluster_indices), n_images)

    cols = 4
    rows = (n_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))

    for i, idx in enumerate(random_samples):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        try:
            img = Image.open(test_paths[idx])
            ax.imshow(img.resize((224, 224)))
            ax.axis('off')
        except:
            ax.set_title("Error")
            ax.axis('off')

    for i in range(n_images, rows * cols):
        row, col = divmod(i, cols)
        ax = axes[row][col] if rows > 1 else axes[col] if cols > 1 else axes
        ax.axis('off')

    plt.suptitle(f"GMM k=6 - Cluster {cluster_id}", fontsize = 14)
    plt.tight_layout()

    # Guardar la figura
    output_path = f"cluster_{cluster_id}_muestras.png"
    plt.savefig(output_path, dpi = 600, bbox_inches = 'tight')

    from google.colab import files
    files.download(output_path)

    plt.show()

    plt.close()

In [ ]:
# Inercia en validación después de predict
gmm_val_preds_k6 = gmm.predict(features_val)
_, val_distances_k6 = pairwise_distances_argmin_min(features_val, gmm.means_)
val_inertia_k6 = np.sum(val_distances_k6 ** 2)
print(f"Inercia en VALIDACIÓN con GMM k = 6: {val_inertia_k6:.2f}")

In [ ]:
# Matriz de confusión
y_true = df[df["Partition"] == "test"]["Species"].tolist()
le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true)
y_pred = gmm_preds_test6

cm = confusion_matrix(y_true_encoded, y_pred)

plt.figure(figsize = (10, 6))
sns.heatmap(cm, annot = True, fmt = "d", cmap = "Blues", xticklabels = range(optimal_k_gmm), yticklabels = le.classes_)
plt.xlabel("Cluster inferido (GMM)")
plt.ylabel("Etiqueta verdadera (especies)")
plt.title("Matriz de confusión - GMM vs etiquetas reales")

# Guardar la matriz de confusión
plt.savefig("matriz_confusion_GMM6.png", dpi = 600, bbox_inches = 'tight')
files.download("matriz_confusion_GMM6.png")

plt.show()

In [ ]:
# Asignación de clase dominante por cluster y Jaccard
assignments = {}
for cluster_id in range(optimal_k_gmm):
    indices = np.where(np.array(y_pred) == cluster_id)[0]
    true_labels_in_cluster = [y_true_encoded[i] for i in indices]
    if true_labels_in_cluster:
        majority_label = max(set(true_labels_in_cluster), key = true_labels_in_cluster.count)
        assignments[cluster_id] = majority_label
    else:
        assignments[cluster_id] = -1

y_pred_mapped = [assignments[c] for c in y_pred]
jaccard_macro = jaccard_score(y_true_encoded, y_pred_mapped, average = "macro")
print(f"Jaccard Index (macro): {jaccard_macro:.4f}")

In [ ]:
# Métricas adicionales: ARI y Fowlkes-Mallows
ari = adjusted_rand_score(y_true_encoded, y_pred)
fmi = fowlkes_mallows_score(y_true_encoded, y_pred)
print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Fowlkes-Mallows Index: {fmi:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

unique_clusters_gmm = np.unique(y_pred)
unique_labels = np.unique(y_true_encoded)
markers = ["o", "s", "v", "^", "<", ">", "d", "*", "P", "X"]

plt.figure(figsize = (10, 7))

for cluster_id in unique_clusters_gmm:
    for label_id in unique_labels:
        indices = np.where((y_pred == cluster_id) & (y_true_encoded == label_id))[0]
        if len(indices) == 0:
            continue
        plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1],
                    label = f"Sp {label_id} / Cl {cluster_id}",
                    s = 8,
                    c = [plt.cm.tab20(label_id / len(unique_labels))],
                    marker = markers[int(cluster_id) % len(markers)],
                    edgecolors = "none")

plt.title("t-SNE (test) - Color: Especie real | Forma: Cluster GMM")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.legend(markerscale = 1.5, fontsize = 8, bbox_to_anchor = (1.05, 1), loc = "upper left")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Nombres reales de especies (orden según codificación)
nombres_especies = {
    0: 'Tinamú',
    1: 'Agutí',
    2: 'Paca',
    3: 'Armadillo',
    4: 'Roedor',
    5: 'Ardilla'
}

# Crear DataFrame con t-SNE + predicciones
df_tsne = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'especie': y_true_encoded,
    'cluster': y_pred
})

plt.figure(figsize = (12, 8))

# Colores por especie real
colores = plt.cm.Dark2(np.linspace(0, 1, len(nombres_especies)))

for idx, especie in enumerate(df_tsne['especie'].unique()):
    df_subset = df_tsne[df_tsne['especie'] == especie]
    plt.scatter(df_subset['x'], df_subset['y'],
                s = 12, alpha = 0.5, color = colores[idx],
                label=nombres_especies.get(especie, f"Especie {especie}"))

# Colocar número del cluster en el centro de cada grupo
for (especie, cluster), group in df_tsne.groupby(['especie', 'cluster']):
    x_med = group['x'].mean()
    y_med = group['y'].mean()
    plt.text(x_med, y_med, str(cluster),
             fontsize = 13, weight = 'bold', ha = 'center', va = 'center',
             color = 'black',
             bbox = dict(facecolor = 'white', alpha = 0.8, edgecolor = 'gray', boxstyle = 'circle'))

plt.title("t-SNE — Color: Especie real  |  Número: Cluster K-Means (k=6)", fontsize = 14)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title = "Especie real", bbox_to_anchor = (1.05, 1), loc = 'upper left', fontsize = 9)
plt.tight_layout()

# Guardar la figura
plt.savefig("tsne_especie-cluster_GMM_6.png", dpi = 300, bbox_inches = 'tight')
files.download("tsne_especie-cluster_GMM_6.png")

plt.show()